# Level Break Out

### Load the data

In [5]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

df = pd.read_csv("RawDataDoNotModify/BTCUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv")
df.rename(columns={'Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'},inplace=True)
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)

df['EMA'] = ta.ema(df.close, length=150)
df.tail()

df=df[0:10000]

### Trend detection

In [6]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [7]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [8]:
window=10
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [9]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [17]:
dfpl = df[8700:8800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [13]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    if (candle <= (backcandles+window)) or (candle+window+1 >= len(df)):
        return 0
    
    localdf = df.iloc[candle-backcandles-window:candle-window] #window must be greater than pivot window to avoid look ahead bias
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    levelbreak = 0
    zone_width = 500
    if len(lows)==3:
        support_condition = True
        mean_low = lows.mean()
        for low in lows:
            if abs(low-mean_low)>zone_width:
                support_condition = False
                break
        if support_condition and (mean_low - df.loc[candle].close)>zone_width*2:
            levelbreak = 1

    if len(highs)==3:
        resistance_condition = True
        mean_high = highs.mean()
        for high in highs:
            if abs(high-mean_high)>zone_width:
                resistance_condition = False
                break
        if resistance_condition and (df.loc[candle].close-mean_high)>zone_width*2:
            levelbreak = 2
    return levelbreak

In [14]:
#df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=15))
df['pattern_detected'] = df.apply(lambda row: detect_structure(row.name, backcandles=60, window=11), axis=1)

In [15]:
df[df['pattern_detected']!=0]

,Local time,open,high,low,close,volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
1135,09.03.2020 00:00:00.000 GMT-0400,8016,8017,7759,7803,52.0508,8833.012312,0,0,NaN,1
1137,09.03.2020 02:00:00.000 GMT-0400,7881,7949,7802,7832,56.0575,8807.311449,0,0,NaN,1
1142,09.03.2020 07:00:00.000 GMT-0400,7838,7838,7713,7753,29.1660,8744.915660,0,0,NaN,1
1143,09.03.2020 08:00:00.000 GMT-0400,7755,7864,7730,7827,27.8051,8732.757837,1,0,NaN,1
1144,09.03.2020 09:00:00.000 GMT-0400,7827,7838,7650,7743,25.0234,8719.648461,1,0,NaN,1
1145,09.03.2020 10:00:00.000 GMT-0400,7742,7783,7646,7673,22.9305,8705.785568,1,0,NaN,1
2039,29.04.2020 16:00:00.000 GMT-0400,8678,8880,8660,8878,64.3586,7577.810046,2,0,NaN,2
2040,29.04.2020 17:00:00.000 GMT-0400,8878,8890,8722,8813,64.5599,7594.170177,2,0,NaN,2
2042,29.04.2020 19:00:00.000 GMT-0400,8709,8744,8662,8734,53.9742,7623.837644,2,0,NaN,2
2043,29.04.2020 20:00:00.000 GMT-0400,8734,8770,8666,8765,59.4340,7638.952377,2,0,NaN,2
